In [6]:
import pandas as pd
from ggplot import *
import gmaps
import os

project_id = "the-d-team-164512"
gmaps.configure(api_key=os.environ['GMAP_API_THE_D_TEAM'])

query = """\
SELECT
  store_postcode_count
  , COUNT(patient_id) AS patient_count
  , FIRST(patient_id) AS first_best_patient
FROM(
  SELECT
    t.Patient_ID AS patient_id
    , GROUP_CONCAT(UNIQUE(s.postcode)) AS store_postcodes
    , COUNT(UNIQUE(s.postcode)) AS store_postcode_count
  FROM [the-d-team-164512:dthon.transactions] t
  JOIN [the-d-team-164512:dthon.stores] s
  ON t.Store_ID = s.Store_ID
  GROUP BY
    patient_id
)
GROUP BY
  store_postcode_count
ORDER BY
  store_postcode_count DESC
"""

df = pd.read_gbq(query, project_id)
df

Waiting for job to complete...


,store_postcode_count,patient_count,first_best_patient
0,35,1,109876
1,33,1,235940
2,27,1,34680
3,26,3,174448
4,25,1,91599
5,24,1,258244
6,23,7,223064
7,22,7,235235
8,21,8,392251
9,20,9,251928


In [4]:
query = """\
SELECT
  t.Patient_ID AS patient_id
  , s.postcode AS postcode
  , pc.lat AS lat
  , pc.lon AS lon
  , t.Prescription_Week AS prescription_week
  , t.Dispense_Week AS dispense_week
FROM [the-d-team-164512:dthon.transactions] t
JOIN [the-d-team-164512:dthon.stores] s
ON t.Store_ID = s.Store_ID
-- LEFT JOIN [the-d-team-164512:dthon.postcodes] p
-- ON s.postcode = p.postcode
LEFT JOIN (
  SELECT STRING(postcode) AS postcode, FIRST(lat) AS lat, FIRST(lon) AS lon
  FROM [the-d-team-164512:dthon.postcodes]
  GROUP BY postcode
  ) pc
ON s.postcode = pc.postcode
WHERE
  patient_id = 109876
ORDER BY
  prescription_week
"""
df_most_prolific = pd.read_gbq(query, project_id)
df_most_prolific.head(3)

Waiting for job to complete...


,patient_id,postcode,lat,lon,prescription_week,dispense_week
0,109876,2517,-34.358093,150.900783,2010-05-16,2011-01-02
1,109876,2517,-34.358093,150.900783,2010-10-17,2011-01-16
2,109876,3076,-37.638363,145.009493,2010-10-31,2011-03-13


In [10]:
locations = df_most_prolific[['lat', 'lon']]
locations = locations.dropna() # presumably due to not found postcode

m = gmaps.Map()
marker_layer = gmaps.marker_layer(locations)
m.add_layer(marker_layer)
m